# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession

session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	8272 candidates
Dev set:	888 candidates
Test set:	4620 candidates


In [3]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

# Computing features

For `SparseLogReg`

In [4]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

%time F_train = featurizer.apply(split=0)
F_train

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 8min 19s, sys: 2.54 s, total: 8min 22s
Wall time: 8min 27s


<8272x122840 sparse matrix of type '<type 'numpy.int64'>'
	with 448906 stored elements in Compressed Sparse Row format>

In [5]:
%%time
F_dev  = featurizer.apply_existing(split=1)
F_test = featurizer.apply_existing(split=2)

Clearing existing...
Running UDF...
[========================================] 100%

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 5min 11s, sys: 1.96 s, total: 5min 13s
Wall time: 5min 16s


Or if already computed:

In [6]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

In [7]:
F_train = featurizer.load_matrix(session, split=0)
F_dev = featurizer.load_matrix(session, split=1)
F_test = featurizer.load_matrix(session, split=2)

# Training `SparseLogReg`

Instead of LSTM, to start.  First, reloading training marginals:

In [8]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [9]:
from snorkel.learning import SparseLogisticRegression
from snorkel.learning.utils import RandomSearch

# Searching over learning rate
param_ranges = {
    'lr' : [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
    'l1_penalty' : [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
    'l2_penalty' : [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
}
model_hyperparams = {
    'n_epochs' : 50,
    'rebalance' : 0.5,
    'print_freq' : 25
}

# NOTE: A larger search (n) would likely lead to a higher score!
searcher = RandomSearch(SparseLogisticRegression, param_ranges, F_train,
                        Y_train=train_marginals, n=5, model_hyperparams=model_hyperparams)

In [10]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

<888x1 sparse matrix of type '<type 'numpy.int64'>'
	with 888 stored elements in Compressed Sparse Row format>

In [11]:
%time disc_model, run_stats = searcher.fit(F_dev, L_gold_dev)

[1] Testing l2_penalty = 1.00e-04, lr = 1.00e-04, l1_penalty = 1.00e-03
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=4740  #epochs=50  batch size=256
[SparseLogisticRegression] Epoch 0 (0.25s)	Average loss=10.617981
[SparseLogisticRegression] Epoch 25 (7.63s)	Average loss=10.511670
[SparseLogisticRegression] Epoch 49 (14.08s)	Average loss=10.487445
[SparseLogisticRegression] Training done (14.08s)
[SparseLogisticRegression] F-1 Score: 0.502242152466
[SparseLogisticRegression] Model saved as <SparseLogisticRegression_0>
[2] Testing l2_penalty = 1.00e-06, lr = 1.00e-05, l1_penalty = 1.00e-03
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=4740  #epochs=50  batch size=256
[SparseLogisticRegression] Epoch 0 (0.24s)	Average loss=10.546364
[SparseLogisticRegression] Epoch 25 (7.34s)	Average loss=10.525524
[SparseLogisticRegression] Epoch 49 (13.78s)	Average loss=10.508618
[SparseLogisticRegression] Training done (13.78s)
[SparseLog

In [12]:
run_stats

,l2_penalty,lr,l1_penalty,Prec.,Rec.,F-1
4,0.000010,0.01000,0.001,0.378086,0.827703,0.519068
2,0.000100,0.00100,0.010,0.378295,0.824324,0.518597
3,0.001000,0.00100,0.010,0.369760,0.834459,0.512448
0,0.000100,0.00010,0.001,0.375839,0.756757,0.502242
1,0.000001,0.00001,0.001,0.319728,0.476351,0.382632


### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

In [13]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')

AnnotatorLabels created: 4620


In [14]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

<4620x1 sparse matrix of type '<type 'numpy.int64'>'
	with 4620 stored elements in Compressed Sparse Row format>

In [15]:
_, _, _, _ = disc_model.error_analysis(session, F_test, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.843
Neg. class accuracy: 0.368
Precision            0.392
Recall               0.843
F1                   0.535
----------------------------------------
TP: 1268 | FP: 1970 | TN: 1145 | FN: 237



# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

**Note: Again, training for more epochs than below will greatly improve performance- try it out!**

In [16]:
from snorkel.learning import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   20,
    'dropout':    0.5,
    'rebalance':  0.25,
    'print_freq': 5
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train, train_marginals, X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)

[reRNN] Training model
[reRNN] n_train=3160  #epochs=20  batch size=256
[reRNN] Epoch 0 (18.71s)	Average loss=0.691763	Dev F1=50.63
[reRNN] Epoch 5 (111.05s)	Average loss=0.652033	Dev F1=45.83
[reRNN] Epoch 10 (203.66s)	Average loss=0.644258	Dev F1=51.26
[reRNN] Epoch 15 (291.96s)	Average loss=0.640916	Dev F1=51.60
[reRNN] Epoch 19 (361.31s)	Average loss=0.639032	Dev F1=50.67
[reRNN] Model saved as <reRNN>
[reRNN] Training done (364.09s)
[reRNN] Loaded model <reRNN>


In [17]:
lstm.score(test, L_gold_test)

(0.40391829929137141, 0.64385382059800667, 0.49641393442622955)